In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #seabornない人はpip installしてね
import os
from datetime import datetime
import numpy as np

# カレントディレクトリを.pyと合わせるために以下を実行
from pathlib import Path
if Path.cwd().name == "notebook":
    os.chdir("..")


# 設定
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 500)
pd.set_option('display.max_columns', 500)
# 浮動小数点数を小数点以下3桁で表示するように設定
pd.set_option('display.float_format', '{:.3f}'.format)


In [2]:
# Mac Matplotlibのデフォルトフォントをヒラギノ角ゴシックに設定
plt.rcParams['font.family'] = 'Hiragino Sans'

In [4]:
# Windows MatplotlibのデフォルトフォントをMeiryoに設定
# plt.rcParams['font.family'] = 'Meiryo'

In [3]:
# point_history と user_info の紐づけ
file_path_point_history = 'data/input/point_history_cleansing.csv'  # ファイルパスを適切に設定してください
df_point_history = pd.read_csv(file_path_point_history)

/Users/naoya.yasuda/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (5,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [23]:
df_point_history.head()

,id,user_id,series_id,shop_id,shop_name,card_id,item_id,amount,amount_kg,unit_id,point,total_point,status,total_amount,coin,rank_id,use_date,created_at,updated_at,id_1,series,super,prefectures,municipality,shop_name_1,address,shop_id_1,created_at_1,updated_at_1,store_opening_time,store_closing_time,rps_opening_time,rps_closing_time,store_latitude,store_longitude
0,22,2023.000,1,52,サン・マルシェ東中田店,superbig047780,1,2,2,0,2,98,1,N,0.000,0,2020-12-17 18:01:24,2020-12-17 18:01:24,2021-04-14 17:57:46,19,1,サン・マルシェ,宮城県,仙台市太白区,東中田店,東中田2-39-27,52,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.2027959,140.9011745
1,27,90.000,1,52,サン・マルシェ東中田店,superbig011187,1,7,7,0,7,143,1,N,21.000,2,2020-12-18 10:59:56,2020-12-18 10:59:56,2020-12-31 13:57:54,19,1,サン・マルシェ,宮城県,仙台市太白区,東中田店,東中田2-39-27,52,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.2027959,140.9011745
2,34,670.000,1,52,サン・マルシェ東中田店,superbig035909,1,5,5,0,5,7,1,N,5.000,0,2020-12-18 14:30:36,2020-12-18 14:30:36,2020-12-31 13:58:58,19,1,サン・マルシェ,宮城県,仙台市太白区,東中田店,東中田2-39-27,52,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.2027959,140.9011745
3,38,725.000,1,52,サン・マルシェ東中田店,superbig016946,1,2,2,0,2,102,1,N,2.000,0,2020-12-18 18:31:55,2020-12-18 18:31:55,2020-12-31 13:59:25,19,1,サン・マルシェ,宮城県,仙台市太白区,東中田店,東中田2-39-27,52,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.2027959,140.9011745
4,39,205.000,1,52,サン・マルシェ東中田店,superbig000727,1,9,9,0,19,1567,1,N,9.000,0,2020-12-18 18:44:35,2020-12-18 18:44:35,2020-12-31 13:56:51,19,1,サン・マルシェ,宮城県,仙台市太白区,東中田店,東中田2-39-27,52,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.2027959,140.9011745


In [24]:
# merge (左内部結合)
df_merge_point = pd.merge(df_point_history, df_user_info, left_on='user_id', right_on='id', how='left')

In [25]:
display(df_merge_point.head(100))
display(df_merge_point.describe())
#print(len(df_merge["user_uid"].unique()))
#print(df_merge.info(verbose=True, show_counts=True))
print(df_merge_point['nickname'].isna().sum())

,id_x,user_id,series_id,shop_id,shop_name,card_id,item_id,amount,amount_kg,unit_id,point,total_point,status,total_amount,coin,rank_id,use_date,created_at,updated_at,id_1,series,super,prefectures,municipality,shop_name_1,address,shop_id_1,created_at_1,updated_at_1,store_opening_time,store_closing_time,rps_opening_time,rps_closing_time,store_latitude,store_longitude,Unnamed: 0,id_y,nickname,club_coin,recycle_point,total_recycle_amount,recycle_amount_per_year,recycle_amount_after_gold_member,rank_continuation_class,gender,birth_day,zipcode,user_uid,チケット獲得合計数
0,22,2023.000,1,52,サン・マルシェ東中田店,superbig047780,1,2,2,0,2,98,1,N,0.000,0,2020-12-17 18:01:24,2020-12-17 18:01:24,2021-04-14 17:57:46,19,1,サン・マルシェ,宮城県,仙台市太白区,東中田店,東中田2-39-27,52,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.2027959,140.9011745,16307.000,2023.000,みみ,100.000,57.000,110.200,34.160,0.000,0.000,男,1978-09-29 00:00:00,9811107.000,nan,nan
1,27,90.000,1,52,サン・マルシェ東中田店,superbig011187,1,7,7,0,7,143,1,N,21.000,2,2020-12-18 10:59:56,2020-12-18 10:59:56,2020-12-31 13:57:54,19,1,サン・マルシェ,宮城県,仙台市太白区,東中田店,東中田2-39-27,52,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.2027959,140.9011745,88794.000,90.000,なおちんママ,2088.000,72.000,771.000,123.100,0.000,0.000,女,1967-08-30 00:00:00,9811107.000,nan,nan
2,34,670.000,1,52,サン・マルシェ東中田店,superbig035909,1,5,5,0,5,7,1,N,5.000,0,2020-12-18 14:30:36,2020-12-18 14:30:36,2020-12-31 13:58:58,19,1,サン・マルシェ,宮城県,仙台市太白区,東中田店,東中田2-39-27,52,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.2027959,140.9011745,64855.000,670.000,ゆみ,146.000,135.000,140.800,0.000,0.000,0.000,女,1968-02-12 00:00:00,9811103.000,nan,nan
3,38,725.000,1,52,サン・マルシェ東中田店,superbig016946,1,2,2,0,2,102,1,N,2.000,0,2020-12-18 18:31:55,2020-12-18 18:31:55,2020-12-31 13:59:25,19,1,サン・マルシェ,宮城県,仙台市太白区,東中田店,東中田2-39-27,52,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.2027959,140.9011745,70536.000,725.000,みい,1595.000,146.000,192.200,0.000,0.000,0.000,女,1988-06-28 00:00:00,9811103.000,nan,nan
4,39,205.000,1,52,サン・マルシェ東中田店,superbig000727,1,9,9,0,19,1567,1,N,9.000,0,2020-12-18 18:44:35,2020-12-18 18:44:35,2020-12-31 13:56:51,19,1,サン・マルシェ,宮城県,仙台市太白区,東中田店,東中田2-39-27,52,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.2027959,140.9011745,16580.000,205.000,Tomoo,831.000,139.000,553.500,111.100,0.000,0.000,女,1970-05-10 00:00:00,9811102.000,205.000,3.000
5,43,205.000,1,52,サン・マルシェ東中田店,superbig000727,1,7,7,0,7,1574,1,N,7.000,0,2020-12-19 11:20:13,2020-12-19 11:20:13,2020-12-31 13:56:43,19,1,サン・マルシェ,宮城県,仙台市太白区,東中田店,東中田2-39-27,52,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.2027959,140.9011745,16580.000,205.000,Tomoo,831.000,139.000,553.500,111.100,0.000,0.000,女,1970-05-10 00:00:00,9811102.000,205.000,3.000
6,60,539.000,1,52,サン・マルシェ東中田店,superbig053591,1,25,25,0,25,98,1,N,25.000,0,2020-12-19 16:49:40,2020-12-19 16:49:40,2020-12-31 14:01:51,19,1,サン・マルシェ,宮城県,仙台市太白区,東中田店,東中田2-39-27,52,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.2027959,140.9011745,51271.000,539.000,櫻井智子,2088.000,243.000,620.500,143.000,0.000,0.000,女,1970-11-04 00:00:00,9820003.000,539.000,1.000
7,61,162.000,1,52,サン・マルシェ東中田店,superbig013909,1,8,8,0,8,166,1,N,8.000,0,2020-12-19 16:53:02,2020-12-19 16:53:02,2020-12-31 14:01:29,19,1,サン・マルシェ,宮城県,仙台市太白区,東中田店,東中田2-39-27,52,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.2027959,140.9011745,12110.000,162.000,otchi1,36.000,122.000,594.500,210.200,71.000,3.000,女,1966-08-13 00:00:00,9811102.000,162.000,41.000
8,82,158.000,1,52,サン・マルシェ東中田店,superbig034486,1,14,14,0,14,150,1,N,42.000,2,2020-12-28 10:46:27,2020-12-28 10:46:27,2020-12-31 14:02:44,19,1,サン・マルシェ,宮城県,仙台市太白区,東中田店,東中田2-39-27,52,2021-09-21 04:25:00,2023-09-20 15:59:08,09:00:00,21:00:00,09:00:00,21:00:00,38.2027959,140.9011745,11693.000,158.000,ゆかりん,1035.000,184.000,1167.000,327.000,0.000,0.000,女,1974-01-1

,id_x,user_id,status,Unnamed: 0,id_y,club_coin,recycle_point,total_recycle_amount,recycle_amount_per_year,recycle_amount_after_gold_member,rank_continuation_class,zipcode,user_uid,チケット獲得合計数
count,2164432.000,2164432.000,2164432.000,2100161.000,2100161.000,2100161.000,2100161.000,2100161.000,2100161.000,2100161.000,2100161.000,1971228.000,1282517.000,1282517.000
mean,11838797.623,41935.219,3.251,49143.071,41193.665,inf,22.137,412.830,241.820,6.143,0.533,7397573.061,46151.249,194.190
std,3910957.833,32714.403,2.802,29562.654,32304.356,nan,77.542,897.034,621.777,21.797,0.984,3176520.803,34706.842,160.480
min,22.000,1.000,1.000,3.000,62.000,0.000,0.000,0.000,0.000,0.000,0.000,1000006.000,77.000,1.000
25%,9636807.500,11003.000,1.000,22204.000,10839.000,282.000,0.000,104.000,73.000,0.000,0.000,3210934.000,11279.000,38.000
50%,12975568.500,35410.000,1.000,47729.000,34488.000,866.000,0.000,218.000,140.000,0.000,0.000,9638051.000,40348.000,180.000
75%,14888394.250,72455.000,7.000,76480.000,70606.000,2072.000,0.000,418.000,225.000,0.000,1.000,9820033.000,82967.000,315.000
max,16576491.000,105518.000,7.000,99819.000,105512.000,inf,3614.000,16880.000,15460.000,149.700,5.000,9999999.000,105512.000,954.000


64636


In [26]:
df_user_info.columns

Index(['Unnamed: 0', 'id', 'nickname', 'club_coin', 'recycle_point',
       'total_recycle_amount', 'recycle_amount_per_year',
       'recycle_amount_after_gold_member', 'rank_continuation_class', 'gender',
       'birth_day', 'zipcode', 'user_uid', 'チケット獲得合計数'],
      dtype='object')